In [ ]:
import pandas as pd
import numpy as np
from os import path as osp
import json
from datetime import datetime
from expense_analysis.tools import get_hash
from loguru import logger

from expense_analysis.correction import (
    CategoryCorrectionWhereLabelContains,
    CategoryCorrectionFromLoc,
    DateCorrectionFromLoc,
    RowDroppingFromLoc,
    CorrectionSet
)

In [ ]:
with open("data_version.json") as f:
    data_version = json.load(f)
    
with open("data_history.json") as f:
    data_history = json.load(f)
    
DATA_SUBDIR = osp.abspath("./datasets-history")

In [ ]:
def safe_contains(s, keyword):
    if pd.isna(s):
        return False
    else:
        return keyword.lower() in s.lower()

In [ ]:
FINAL_COLS = [
    "date",
    "bank_name",
    "custom_account_name",
    "label",
    "amount",
    "category_name",
    "proportion",
    "to_be_matched",
]

# 22/08

## From updated Cozy Cloud

In [ ]:
DF_NAME = "cozy_cloud"
raw_data_path = osp.abspath("../../perso/tmp/banks/export-data-banks-3-month-2024.csv")
df_new_cc_raw = pd.read_csv(raw_data_path, sep=";", decimal=",")
df_new_cc = df_new_cc_raw.rename(columns={c: c.lower().replace(" ", "_") for c in df_new_cc_raw.columns})
df_new_cc["date"] = pd.to_datetime(df_new_cc["date"], format="%Y-%m-%d")
df_new_cc = df_new_cc.assign(proportion=1.0, shared=False)
df_new_cc.sort_values(by="date", inplace=True)
df_new_cc["amount"] = df_new_cc.amount.astype(float)
df_new_cc = df_new_cc[np.abs(df_new_cc.amount) > 0.01]
df_new_cc["to_be_matched"] = False

corrections_set = CorrectionSet.from_json(f"corrections_{DF_NAME}.json")

corrections_set.apply(df_new_cc)

df_new_cc = df_new_cc[
    np.logical_or(
        ("2024-04-01" <= df_new_cc.date) * (df_new_cc.date < "2024-09-01"),
        ("2022-04-01" <= df_new_cc.date) * (df_new_cc.date < "2022-09-01"),
    )
]

df_new_cc = df_new_cc[FINAL_COLS].reset_index(drop=True).reset_index()

# check version
DF_ = df_new_cc
raw_data_path_entry_name = f"{DF_NAME}_raw_data_path"
hash_entry_name = f"{DF_NAME}_hash"
update_date_entry_name = f"{DF_NAME}_last_update"
df_hash = get_hash(DF_)
if df_hash != data_version.get(hash_entry_name) or raw_data_path != data_version.get(raw_data_path_entry_name):
    logger.warning("Changes in df_new_cc: updating data_version.json")
    data_version[hash_entry_name] = df_hash
    data_version[raw_data_path_entry_name] = raw_data_path
    data_version[update_date_entry_name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("data_version.json", "w") as f:
        json.dump(data_version, f, indent=4)
        
    # update this dataset's version
    DF_.to_csv(osp.join(DATA_SUBDIR, f"{DF_NAME}_{df_hash}.csv"), index=False)

df_new_cc.sample(3)

## From food cards

### EdenRed : no information on date of interrest

In [ ]:
DF_NAME = "myedenred"
raw_data_path = osp.abspath("../../perso/tmp/banks/MyEdenRed-2023.csv")
df_edenred = pd.read_csv(raw_data_path, sep=",", decimal=",")
df_edenred = df_edenred.drop(columns=["Unnamed: 0", "Statut", "Type"])
df_edenred = df_edenred.rename(columns={"Date": "date", "Détails": "label", "Montant": "amount"})
df_edenred["date"] = pd.to_datetime(df_edenred["date"], format="%d/%m/%Y")
df_edenred["amount"] = df_edenred.amount.apply(lambda x: x.replace("€", "").replace(",", ".")).astype(float)
df_edenred["bank_name"] = "EdenRed"
df_edenred["custom_account_name"] = "EdenRed"
df_edenred["category_name"] = "restaurantsAndBars"
df_edenred["proportion"] = 1.0
df_edenred["to_be_matched"] = True

# pre fill some categories
corrections_set = CorrectionSet.from_json(f"corrections_{DF_NAME}.json")
corrections_set.apply(df_edenred)

# duplicate it to insert equivalent "salary" entries
df_edenred = pd.concat(
    [
        # df_edenred.assign(categrory_name="activityIncome"),
        df_edenred.assign(categrory_name="supermarket", amount=-df_edenred.amount)
    ]
).sort_values(by="date")

# filter date of interrest
df_edenred = df_edenred[
    np.logical_or(
        ("2024-04-01" <= df_edenred.date) * (df_edenred.date < "2024-09-01"),
        ("2022-04-01" <= df_edenred.date) * (df_edenred.date < "2022-09-01"),
    )
]

df_edenred = df_edenred[FINAL_COLS].reset_index(drop=True).reset_index()

# check version
DF_NAME = "myedenred"
DF_ = df_edenred
raw_data_path_entry_name = f"{DF_NAME}_raw_data_path"
hash_entry_name = f"{DF_NAME}_hash"
update_date_entry_name = f"{DF_NAME}_last_update"
df_hash = get_hash(DF_.to_dict())
if df_hash != data_version.get(hash_entry_name) or raw_data_path != data_version.get(raw_data_path_entry_name):
    logger.warning("Changes in df_new_cc: updating data_version.json")
    data_version[hash_entry_name] = df_hash
    data_version[raw_data_path_entry_name] = raw_data_path
    data_version[update_date_entry_name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("data_version.json", "w") as f:
        json.dump(data_version, f, indent=4)
        
    # update this dataset's version
    DF_.to_csv(osp.join(DATA_SUBDIR, f"{DF_NAME}_{df_hash}.csv"), index=False)

try:
    df_edenred.sample(3)
except ValueError:
    pass

### Swile

In [ ]:
DF_NAME = "swile"

raw_data_path = osp.abspath("../../perso/tmp/banks/swile_2024_08.csv")
df_swile = pd.read_csv(raw_data_path, sep=";")
df_swile["date"] = pd.to_datetime(df_swile["date"], format="%Y/%m/%d")
df_swile["bank_name"] = "Swile"
df_swile["custom_account_name"] = "Swile"
df_swile["proportion"] = np.where(df_swile.category_name == "supermarket", 0.7, 1.0)

df_swile["to_be_matched"] = np.where(df_swile.amount < -25, True, False)

df_swile = df_swile[
    np.logical_or(
        ("2024-04-01" <= df_swile.date) * (df_swile.date < "2024-09-01"),
        ("2022-04-01" <= df_swile.date) * (df_swile.date < "2022-09-01"),
    )
]

df_swile = df_swile[FINAL_COLS].reset_index(drop=True).reset_index()

# check version
DF_ = df_swile
raw_data_path_entry_name = f"{DF_NAME}_raw_data_path"
hash_entry_name = f"{DF_NAME}_hash"
update_date_entry_name = f"{DF_NAME}_last_update"
df_hash = get_hash(DF_.to_dict())
if df_hash != data_version.get(hash_entry_name) or raw_data_path != data_version.get(raw_data_path_entry_name):
    logger.warning("Changes in df_new_cc: updating data_version.json")
    data_version[hash_entry_name] = df_hash
    data_version[raw_data_path_entry_name] = raw_data_path
    data_version[update_date_entry_name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("data_version.json", "w") as f:
        json.dump(data_version, f, indent=4)
        
    # update this dataset's version
    DF_.to_csv(osp.join(DATA_SUBDIR, f"{DF_NAME}_{df_hash}.csv"), index=False)

df_swile.sample(3)

## From Cospender

In [ ]:
# READ
DF_NAME = "cospender"
corrections_set = CorrectionSet.from_json(f"corrections_{DF_NAME}.json")
raw_data_path = "/Users/francois.weber/perso/tmp/banks/cospender-Lionciole-2024-08.csv"
df_cospend_raw = pd.read_csv(raw_data_path, sep=";", decimal=",")

df_cospend = df_cospend_raw.rename(columns={c: c.lower().replace(" ", "_") for c in df_cospend_raw.columns})
df_cospend["date"] = pd.to_datetime(df_cospend["date"], format="%Y%m%d")
df_cospend = df_cospend.sort_values(by="date")
df_cospend["amount"] = -df_cospend["amount"].abs()
df_cospend["transaction"] = "Payed by: " + df_cospend["member"] + " for " + df_cospend["beneficiaries"]

df_cospend = df_cospend.rename(columns={"description": "label"}).assign(
    bank_name="Cospender", custom_account_name=df_cospend.member, category_name=None
)
df_cospend = df_cospend.assign(
    bank_name="Cospender",
    custom_account_name=df_cospend.transaction,
    category_name=None,
)


# try to auto guess repartition
df_cospend["proportion"] = 1.0
# 70/30 before going to 5 rue des Juifs
df_cospend.loc[(df_cospend.date < "2022-09-01") * (df_cospend.beneficiaries == "All"), "proportion"] = 0.7
# 60/40 before going to 5 rue des Juifs
df_cospend.loc[
    ("2023-02-01" <= df_cospend.date) * (df_cospend.beneficiaries == "All"),
    "proportion",
] = 0.6

df_cospend.loc[df_cospend.beneficiaries == "François", "amount"] = -df_cospend.loc[
    df_cospend.beneficiaries == "François", "amount"
]
df_cospend.loc[df_cospend.beneficiaries == "François", "category_name"] = "friendBorrowing"
df_cospend.loc[df_cospend.beneficiaries == "louloute", "category_name"] = "friendBorrowing"

df_cospend = df_cospend[np.abs(df_cospend["amount"]) > 0.01]

# Correctionss
corrections_set.apply(df_cospend)


df_cospend = df_cospend.reset_index(drop=True).reset_index()

### Huge manual corrections
corrections_set = CorrectionSet.from_json(f"corrections_{DF_NAME}.json")
corrections_set.apply(df_cospend)
    
    
# every time I log an expenses in Cospender, I'm also charged from the bank. let's keep this one and remove the one in the bank account
# BUT, sometimes the cospend amount does not match the real-life expense
# SO, rather than erasing it bank line, just add a positive amount somewhere that will cancel the bank expense in a groupby
df_cospend_cancel_bank_lines = df_cospend[(df_cospend.member == "François")].copy()
df_cospend_cancel_bank_lines["bank_name"] = "cancel from Cospender"
df_cospend_cancel_bank_lines["custom_account_name"] = "Payed by François"
df_cospend_cancel_bank_lines["amount"] = -df_cospend_cancel_bank_lines["amount"]
df_cospend_cancel_bank_lines["proportion"] = 1.0

# posterior modification
df_cospend_cancel_bank_lines.loc[1094, "amount"] = 50 # and not 70

df_cospend = pd.concat([df_cospend, df_cospend_cancel_bank_lines]).sort_values(by="date")
# No longer prepare a flag for expenses that should match something in my other accounts thanks to the trick above
df_cospend["to_be_matched"] = False

# filter date of interrest
df_cospend = df_cospend[
    np.logical_or(
        ("2024-04-01" <= df_cospend.date) * (df_cospend.date < "2024-09-01"),
        ("2022-04-01" <= df_cospend.date) * (df_cospend.date < "2022-09-01"),
    )
]


# # # 
df_cospend = df_cospend[FINAL_COLS].reset_index(drop=True).reset_index()


# check version
DF_ = df_cospend
raw_data_path_entry_name = f"{DF_NAME}_raw_data_path"
hash_entry_name = f"{DF_NAME}_hash"
update_date_entry_name = f"{DF_NAME}_last_update"
df_hash = get_hash(DF_.to_dict())
if df_hash != data_version.get(hash_entry_name) or raw_data_path != data_version.get(raw_data_path_entry_name):
    logger.warning("Changes in df_new_cc: updating data_version.json")
    data_version[hash_entry_name] = df_hash
    data_version[raw_data_path_entry_name] = raw_data_path
    data_version[update_date_entry_name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("data_version.json", "w") as f:
        json.dump(data_version, f, indent=4)
        
    # update this dataset's version
    DF_.to_csv(osp.join(DATA_SUBDIR, f"{DF_NAME}_{df_hash}.csv"), index=False)

df_cospend.sample(3)

## Add recurring expenses from spreadsheet
On 5 rue des Juifs

In [ ]:
# rue des Juifs
DF_NAME = "recurring_expenses_juifs"
raw_data_path = "/Users/francois.weber/code/expense-analysis/recurrent_expenses_rue_des_juifs.csv"
df_spreadsheet_juifs = pd.read_csv(raw_data_path)


DF_ = df_spreadsheet_juifs
raw_data_path_entry_name = f"{DF_NAME}_raw_data_path"
hash_entry_name = f"{DF_NAME}_hash"
update_date_entry_name = f"{DF_NAME}_last_update"
df_hash = get_hash(DF_.to_dict())
if df_hash != data_version.get(hash_entry_name) or raw_data_path != data_version.get(raw_data_path_entry_name):
    logger.warning("Changes in df_new_cc: updating data_version.json")
    data_version[hash_entry_name] = df_hash
    data_version[raw_data_path_entry_name] = raw_data_path
    data_version[update_date_entry_name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("data_version.json", "w") as f:
        json.dump(data_version, f, indent=4)
        
    # update this dataset's version
    DF_.to_csv(osp.join(DATA_SUBDIR, f"{DF_NAME}_{df_hash}.csv"), index=False)

df_spreadsheet_juifs.sample(3)

## Reshape recurring expenses from rue du jeu des enfants
380€ was the sum of every components and it's possible to go finer grain

In [ ]:
raw_data_path = "/Users/francois.weber/code/expense-analysis/reccurent_expenses_rue_du_jeu_des_enfants.csv"
df_spreadsheet_enfants = pd.read_csv(raw_data_path)


DF_NAME = "recurring_expenses_jeux_des_enfants"
DF_ = df_spreadsheet_enfants
raw_data_path_entry_name = f"{DF_NAME}_raw_data_path"
hash_entry_name = f"{DF_NAME}_hash"
update_date_entry_name = f"{DF_NAME}_last_update"
df_hash = get_hash(DF_.to_dict())
if df_hash != data_version.get(hash_entry_name) or raw_data_path != data_version.get(raw_data_path_entry_name):
    logger.warning("Changes in df_new_cc: updating data_version.json")
    data_version[hash_entry_name] = df_hash
    data_version[raw_data_path_entry_name] = raw_data_path
    data_version[update_date_entry_name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("data_version.json", "w") as f:
        json.dump(data_version, f, indent=4)
        
    # update this dataset's version
    DF_.to_csv(osp.join(DATA_SUBDIR, f"{DF_NAME}_{df_hash}.csv"), index=False)

df_spreadsheet_enfants.sample(3)

# Merge them

In [ ]:
raw_data_path = None
df = pd.concat(
    [
        df_new_cc,
        df_cospend,
        df_swile,
        df_edenred,
        df_spreadsheet_juifs,
        df_spreadsheet_enfants,
    ]
)

df["date"] = pd.to_datetime(df["date"])
    
df = (
    df
    .sort_values(by="date")
    .reset_index(drop=True)
    .reset_index(names="uid")
)

df["category_name"] = df["category_name"].str.strip()

df = df[
    np.logical_or(
        ("2024-04-01" <= df.date) * (df.date < "2024-09-01"),
        ("2022-04-01" <= df.date) * (df.date < "2022-09-01"),
    )
]

with open("./cozy_categories_remapping.json") as f:
    category_mapping = json.load(f)
    
df = df.rename(columns={"category_name": "old_category_name"})
df["main_category"] = df["old_category_name"].replace({k: v["mother_category"] for k, v in category_mapping.items()})
df["category_name"] = df["old_category_name"].replace({k: (v["mother_category"] + "-" + v["renamed_category"]) for k, v in category_mapping.items()})

bank_name_to_exclude = ["CCF", "La Banque Postale"] 
df = df[~df.bank_name.isin(bank_name_to_exclude)]

COMPRESS_DATASET = False
if COMPRESS_DATASET:
    # compress dataset
    data = []
    rows = df.to_dict(orient="records")
    prev_row = rows[0]
    trailing = True
    for row in df.to_dict(orient="records")[1:]:
        if row["uid"] != prev_row["uid"] and row.get("date") == prev_row.get("date") and row.get("bank_name") == prev_row.get("bank_name"):
            prev_row["amount"] += row["amount"]
            trailing = True
        else:
            data.append(prev_row)
            prev_row = row
            trailing = False
    if trailing:
        data.append(prev_row)
        
    df = pd.DataFrame(data)

df["year"] = df.date.dt.year
df["month"] = df.date.dt.month
df["shared"] = np.where(df.proportion < 1.0, "share", "perso")
df["real_amount"] = df.amount * df.proportion
df["week"] = df.date.dt.isocalendar().week

DF_NAME = "final_df"
DF_ = df
raw_data_path_entry_name = f"{DF_NAME}_raw_data_path"
hash_entry_name = f"{DF_NAME}_hash"
update_date_entry_name = f"{DF_NAME}_last_update"
df_hash = get_hash(DF_.to_dict())
if df_hash != data_version.get(hash_entry_name) or raw_data_path != data_version.get(raw_data_path_entry_name):
    logger.warning(f"Changes in {DF_NAME}: updating data_version.json")
    data_version[hash_entry_name] = df_hash
    data_version[raw_data_path_entry_name] = raw_data_path
    data_version[update_date_entry_name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("data_version.json", "w") as f:
        json.dump(data_version, f, indent=4)
        
    # update this dataset's version
    DF_.to_csv(osp.join(DATA_SUBDIR, f"{DF_NAME}_{df_hash}.csv"), index=False)

    # maybe update history
    previous_hash = data_history["version_history"][0][hash_entry_name]
    
    data_history["version_history"].insert(0, data_version)
    with open("data_history.json", "w") as f:
        json.dump(data_history, f, indent=4)
    logger.info("Dataset history updated")

df.to_csv("expenses-full-2022-2024.csv", sep=";", index=False)
df.sample(3)

# Feedback